<a href="https://colab.research.google.com/github/Edward-TL/amazon_scraper/blob/master/GENERATOR_ProductDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


#imports

In [6]:
#For the system
import os

#Manage of time
from datetime import datetime, timedelta
from pytz import timezone
import time
import re

#Manage of files
import pandas as pd
import csv

#scrap
from bs4 import BeautifulSoup
from openpyxl.workbook import Workbook
import requests

##Ubicación

In [7]:
%cd 'drive/My Drive/Colab Notebooks/4SS/4SS_db/testing/'
!ls

/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing
'Day Update'	 Masters   Products
'Edition Test'	 Origin   'Untitled document.gdoc'


#Centered Len

In [9]:
def centered_len(max_len):
    def centered_string(string):
        if type(string) != 'string':
            string = str(string)
            
        rest = max_len - len(string)
        n_spaces = int(rest/2)

        string_spaces = ' ' * n_spaces
        
        centered_string = string_spaces + string + string_spaces

        if len(centered_string)<max_len:
            centered_string = ' ' + centered_string

        return centered_string
    return centered_string

# Preload DBs

In [10]:
def preload_dbs(dir):

    '''Recive a directory with a group of dbs and returns an info array
    info[0] = array of DB's names
    info[1] = largest_len_db, for centered'''
    
    databases = os.listdir(dir)
    n_databases = len(databases)

    #initialize
    df = [None]*n_databases
    largest_len_db = 0
    n = 0

    for file_name in databases:
        db_name = f'{file_name}'
        df[n] = db_name
        if len(db_name) > largest_len_db:
            largest_len_db = len(db_name)

        n = n + 1

    info = [df, largest_len_db]
    return info

# Print info Headers

In [24]:
def print_info_headers(len: int):
    largest_len = info[1]
    centered_db = centered_len(largest_len)
    centered_header = centered_db('masterdbs')
    dashes = '-'*largest_len + '-'

    print(f' country | {centered_header}  | Edition')
    print(f' ------- | {dashes} | ------ ')

# Editor Functions

## Count if

In [30]:
def count_if(array, value):
    count = 0
    for element in array:
        if element == value:
            count += 1
    return count

## Column To Set

In [28]:
def column_to_set(df, df_column, remove_None=True, sort=False, raw_data=False):
    raw_info = df[df_column].tolist()
    #Remember, this gets de None Value
    array = list(set(raw_info))

    if remove_None == True:
        None_cases = count_if(raw_info, None)
        if None_cases:
            array.remove(None)

    if sort == True:
        array = sorted(array)

    if raw_data == True:
        return array, raw_info
    
    else:
        return array

## Gen Product DB

In [32]:
def gen_product_db(df):

    #Get arrays
    time_array = df['time'].to_list()
    ranks = df['Rank'].to_list()
    product_names = df['Product Names'].to_list()
    product_IDs = df['Product ID'].to_list()
    images_id = df['Image ID'].to_list()
    stars = df['Stars'].to_list()
    reviews = df['Reviews'].to_list()
    authors = df['Authors/Company'].to_list()
    editions = df['Edition/Console'].to_list()
    prices_1 = df['Price_std_or_min'].to_list()
    max_prices = df['Max_prices'].to_list()

    #From column name main info
    products_list = column_to_set(df, 'Product Names', sort=False)
    start = (len(product_names) - 1)
    n_header = 0
    
    #Generate de Columns for the main info
    product_ID = [None] * len(products_list)
    product_url = [None] * len(products_list)
    product_rank_image = [None] * len(products_list)
    product_image = [None] * len(products_list)
    last_date = [None] * len(products_list)
    rank = [None] * len(products_list)
    n_stars = [None] * len(products_list)
    n_reviews = [None] * len(products_list)
    price = [None] * len(products_list)
    max_price = [None] * len(products_list)
    last_review = [None] * len(products_list)
    Description = [None] * len(products_list)

    #Check from the bottom 
    for product in range(len(products_list)):
        product_name = products_list[product]
        check = start

        while product_names[check] != product_name:
            check -= 1
            if check < 0:
                print(len(product_names), check, product, product_name)
                break

        product_ID[product] = product_IDs[check]
        product_rank_image[product] = images_id[check]
        last_date[product] = time_array[check]
        rank[product] = ranks[check]
        n_stars[product] = stars[check]
        n_reviews[product] = reviews[check]
        price[product] = prices_1[check]
        max_price[product] = max_prices[check]

    #info to dictionary
    headers = ['Product Name', 'Product ID', 'Product URL', 'Image ID (ranking page)', 'Image ID (product page)',
            'Last Update', 'Rank', 'Stars','Reviews', 'Price (Min or Mean)', 'Max Price', 'Last Principal Review - Title', 'Last Principal Review', 'Last Star Rate', 'Product Description']
    info = {headers[0] : products_list,
            headers[1] : product_ID,
            headers[2] : product_url,
            headers[3] : product_rank_image,
            headers[4] : product_image,
            headers[5] : last_date,
            headers[6] : rank,
            headers[7] : n_stars,
            headers[8] : n_reviews,
            headers[9] : price,
            headers[10] : max_price,
            headers[11] : last_review,
            headers[12] : Description,
            }
    
    product_db = pd.DataFrame.from_dict(info)

    return product_db

In [ ]:
# @replace_urls_with_ids
# def edit_function(db_to_edit):
#     return

# Main Edition

In [25]:
def main_edition(info,original_dir, dir_to_send, country=None):

    centered_db = centered_len(info[1])

    df_names = info[0]
    
    for n in range(len(df_names)):

        if country:
            print(f'   {country}   ', end=' | ')

        #file name extract
        parquet_len = len('.parquet')
        file_name_len = len(df_names[n])-parquet_len
        file_name = df_names[n][0:file_name_len]

        #File name print to log
        print(f' {centered_db(file_name)}', end=' | ')

        #--------Edition--------
        #Load Dataframe
        db_dir = f'{original_dir}/{df_names[n]}'
        db_to_edit = pd.read_parquet(db_dir)
        

        edited_db = gen_product_db(db_to_edit)
        #--------Upadting--------
        #File names
        csv_file = f'{dir_to_send}/csv/{file_name}.csv'
        excel_file = f'{dir_to_send}/excel/{file_name}.xlsx'
        parquet_file = f'{dir_to_send}/parquet/{file_name}.parquet'
        #File creation
        edited_db.to_csv(csv_file,sep="|")
        edited_db.to_excel(excel_file, index=False)
        edited_db.to_parquet(parquet_file, allow_truncated_timestamps=True)

        print('Done')

# Deploy


In [33]:
secure = True

if secure == True:
    countries = ['mx', 'br']

    for country in countries:
        scrap_dir = f'Masters/Parquet backup/{country}/parquet'
        product_db_dir = f'Products/{country}'
        info = preload_dbs(scrap_dir)
        print_info_headers(info[1])
        main_edition(info, original_dir=scrap_dir, dir_to_send=product_db_dir, country=country)

 country |                 masterdbs                 | Edition
 ------- | ----------------------------------------- | ------ 
   mx    |           mx-master_db_videogames         | Done
   mx    |              mx-master_db_toys            | Done
   mx    |             mx-master_db_tools            | Done
   mx    |             mx-master_db_sports           | Done
   mx    |            mx-master_db_software          | Done
   mx    |             mx-master_db_shoes            | Done
   mx    |          mx-master_db_pet-supplies        | Done
   mx    |         mx-master_db_officeproduct        | Done
   mx    |      mx-master_db_musical-instruments     | Done
   mx    |            mx-master_db_kitchen           | Done
   mx    |             mx-master_db_music            | Done
   mx    |           mx-master_db_industrial         | Done
   mx    |              mx-master_db_hpc             | Done
   mx    |            mx-master_db_handmade          | Done
   mx    |            mx-master_db